In [1]:
import itertools
import re
import numpy as np
import pandas as pd
import openai
import pandas as pd
import itertools
import streamlit as st
import pickle
# Anna's imports
from transformers import FlaxAutoModelForSeq2SeqLM
from transformers import AutoTokenizer
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from gradio_client import Client
from transformers import pipeline
from groq import Groq


In [2]:
def combinations_of_two(ingredients_input): ###dealt with the issue of missing space crash

    '''
    The function generates all unique pairs of ingredients that can be made from the input list of ingredients.
    NOTE FOR FRONT-END: The output of this function is the input for data_query()
    '''

    ingredients_combinations = []
    powerpowerset = []
    ingredients = re.split(r',', ingredients_input.strip())
    ingredients_list = list(set(ingredient.strip() for ingredient in ingredients))
    for r in range(len(ingredients_list)+1):
        combinations = itertools.combinations(ingredients_list, r)
        #powerset.extend(subset for subset in combinations if len(subset) > 1)
        for comb in combinations:
            if len(comb) > 1:
                if len(comb) < 3:
                    ingredients_combinations.append(comb)
                else:
                    powerpowerset.append(comb)
                    for power in powerpowerset:
                        lowerset = []
                        combins = itertools.combinations(power, 2)
                        for arrange in combins:
                            lowerset.append(arrange)
                    ingredients_combinations.append(lowerset)
    return ingredients_combinations

combinations = combinations_of_two('chicken, onion, garlic, cheese')

In [5]:
df = pd.read_parquet('Halved-DF.parquet.gzip')

In [6]:
def data_query(df, ingredients_combinations): ##Added a penalty of -5 for pairings that are not in the dataframe
    data = []
    for combination in ingredients_combinations:
        if len(combination) < 3:
            ingredient1, ingredient2 = combination
            query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
            score = df.query(query_str)['scaled_col'].values
            if len(score) > 0:
                data.append({'Combination': combination, 'Score': score})
            else:
                continue
        else:
            scores = []
            for i in combination:
                ingredient1, ingredient2 = i
                query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
                score = df.query(query_str)['scaled_col'].values
                if len(score) > 0:
                    scores.append(score[0])
                else:
                    scores.append(-5)
            data.append({'Combination': combination, 'Score': scores})

    df_comb = pd.DataFrame(data)
    return df_comb

df_comb = data_query(df, combinations)
df_comb

,Combination,Score
0,"(garlic, cheese)",[1.0944496012124034]
1,"(garlic, onion)",[1.4235409132992736]
2,"(garlic, chicken)",[1.1800811339314616]
3,"(cheese, onion)",[1.4120038875676597]
4,"(cheese, chicken)",[1.0177501859513405]
5,"(onion, chicken)",[1.3248796627107513]
6,"[(garlic, cheese), (garlic, onion), (cheese, o...","[1.0944496012124034, 1.4235409132992736, 1.412..."
7,"[(garlic, cheese), (garlic, chicken), (cheese,...","[1.0944496012124034, 1.1800811339314616, 1.017..."
8,"[(garlic, onion), (garlic, chicken), (onion, c...","[1.4235409132992736, 1.1800811339314616, 1.324..."
9,"[(cheese, onion), (cheese, chicken), (onion, c...","[1.4120038875676597, 1.0177501859513405, 1.324..."


In [8]:
def muse_comb(data_query_df): ###If this takes too long, consider taking the nested calculate_sum(array) outside of the function
    '''
     the function calculates the sum of the "Score" values and returns the three combinations with the largest sums
     OUTPUT: [['yeast', 'butter', 'eggs', 'pepper', 'cabbage', 'pork', 'flour', 'sugar'],
                 ['butter', 'eggs', 'pepper', 'cabbage', 'pork', 'flour', 'sugar'],
                 ['yeast', 'butter', 'eggs', 'pepper', 'cabbage', 'flour', 'sugar']]

     NOTE FOR FRONT-END: The return is a list of lists so access the values by indexing e.g. output[0]

                         The output of this function is the input for the recipe generator

                         We might need a function to convert each lists into strings if
                         the recipe generator doesn't do this automatically.
    '''

    def calculate_sum(array):
        return sum(array)

    def ingredients_to_lists(lists):
        ingredients_list = []
        for i in range(3):
            tmp_list = []
            for x in lists[i]:
                tmp_list.append(x[0])
                tmp_list.append(x[1])
            ingredients_list.append(list(set(tmp_list)))

        return ingredients_list

    for i in range(len(data_query_df)):
        data_query_df["Sum"] = data_query_df["Score"].apply(calculate_sum)

    max_values = data_query_df.nlargest(3, "Sum")

    max_values = max_values["Combination"].reset_index(drop=True)

    ingredients_lists = ingredients_to_lists(max_values)

    return ingredients_lists


In [9]:
lists = muse_comb(df_comb)
lists

[['garlic', 'cheese', 'onion', 'chicken'],
 ['garlic', 'cheese', 'onion'],
 ['garlic', 'onion', 'chicken']]

In [67]:
api_key = "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"
client = Groq(
api_key=api_key
)
recipe_list = []
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"Suggest one recipe with {lists[0]} only. The final format of the output should contain Title, Ingredients and Directions only",
        }
    ],
    model="llama3-8b-8192",
)

recipe = chat_completion.choices[0].message.content
recipe

2024-05-07 16:17:58.273 INFO    httpx: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


'Here is a recipe that combines the given ingredients:\n\n**Garlic Chicken Fritters**\n\n**Ingredients:**\n\n* 1 lb boneless, skinless chicken breast, cut into small pieces\n* 2 cloves garlic, minced\n* 1 medium onion, finely chopped\n* 1 cup shredded cheese (cheddar or mozzarella work well)\n* Salt and pepper to taste\n\n**Directions:**\n\n1. Preheat oven to 375°F (190°C).\n2. In a bowl, combine chicken, garlic, onion, and cheese.\n3. Mix well to combine.\n4. Form into small patties (about 1 inch in diameter).\n5. Place on a baking sheet lined with parchment paper.\n6. Bake for 15-20 minutes or until cooked through.\n7. Serve hot and enjoy!'

In [69]:
parts = recipe.split("**")
title = parts[1].strip()
ingredients = parts[4].strip()
directions = parts[6].strip()

recipe_dict = {}
recipe_dict['title'] = title
recipe_dict['ingredients'] = ingredients
recipe_dict['directions'] = directions
recipe_dict

{'title': 'Garlic Chicken Fritters',
 'ingredients': '* 1 lb boneless, skinless chicken breast, cut into small pieces\n* 2 cloves garlic, minced\n* 1 medium onion, finely chopped\n* 1 cup shredded cheese (cheddar or mozzarella work well)\n* Salt and pepper to taste',
 'directions': '1. Preheat oven to 375°F (190°C).\n2. In a bowl, combine chicken, garlic, onion, and cheese.\n3. Mix well to combine.\n4. Form into small patties (about 1 inch in diameter).\n5. Place on a baking sheet lined with parchment paper.\n6. Bake for 15-20 minutes or until cooked through.\n7. Serve hot and enjoy!'}

In [70]:
def groq_recipe_generator(lists):
    api_key = "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"
    client = Groq(
    api_key=api_key
    )
    recipe_list = []
    for i in range(len(lists)):
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"Suggest one recipe with {lists[0]} only. The final format of the output should contain Title, Ingredients and Directions only",
                }
            ],
            model="llama3-8b-8192",
        )

        recipe = chat_completion.choices[0].message.content

        parts = recipe.split("**")
        title = parts[1].strip()
        ingredients = parts[4].strip()
        directions = parts[6].strip()

        recipe_dict = {}
        recipe_dict['title'] = title
        recipe_dict['ingredients'] = ingredients
        recipe_dict['directions'] = directions

        recipe_list.append(recipe_dict)

    return recipe_list


In [71]:
groq_recipe_generator(lists)

2024-05-07 16:20:09.792 INFO    httpx: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-07 16:20:10.261 INFO    httpx: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-07 16:20:10.815 INFO    httpx: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


[{'title': 'Garlic Chicken Crostini',
  'ingredients': '* 1 pound boneless, skinless chicken breast, cut into small pieces\n* 3 cloves garlic, minced\n* 1 medium onion, finely chopped\n* 1 cup shredded cheese',
  'directions': '1. Preheat oven to 375°F (190°C).\n2. In a skillet, sauté chicken, garlic, and onion until chicken is cooked through.\n3. Arrange toasted bread slices on a baking sheet.\n4. Top each bread slice with a spoonful of the chicken mixture and sprinkle with cheese.\n5. Bake for 10-12 minutes or until cheese is melted and bubbly.\n6. Serve warm and enjoy!'},
 {'title': 'Garlic Chicken Cakes',
  'ingredients': '* 1 lb boneless, skinless chicken breast or thighs, cut into small pieces\n* 2 cloves of garlic, minced\n* 1 small onion, finely chopped\n* 1 cup shredded cheese\n* Salt and pepper to taste',
  'directions': '1. Preheat oven to 375°F (190°C).\n2. In a large bowl, combine chicken, garlic, and onion. Mix well.\n3. Add shredded cheese and mix until well combined.\n4